In [1]:
import pandas as pd
import sys
import os

# Asegurarse de que src/ esté en el path
sys.path.append(os.path.abspath('../src'))

In [2]:
import sys
import os

# Ruta absoluta al root del proyecto
project_root = os.path.abspath('.')
src_path = os.path.join(project_root, 'src')

if src_path not in sys.path:
    sys.path.append(src_path)

In [4]:
# Ejecutar todo el pipeline como si corrieras main.py
from main import main

main()

ModuleNotFoundError: No module named 'src'